## DDB Context

In [ ]:
# Stack prefix used for ddb stack
stack_prefix = 'xt0-'

In [ ]:
# DDB stack
ddb_stack = stack_prefix + 'sddb'

In [ ]:
# AWS SDK import and client for cloud formation
import boto3

client = boto3.client('cloudformation')

In [ ]:
# We need the name of the model instance table
response = client.describe_stacks(
    StackName=ddb_stack
)

outputs = response['Stacks'][0]['Outputs']
print outputs

model_instance_table_arn =  [o for o in outputs if o['OutputKey'] == 'InstanceTableArn'][0]['OutputValue']
print model_instance_table_arn
model_instance_table_name = [o for o in outputs if o['OutputKey'] == 'InstanceTableName'][0]['OutputValue']
print model_instance_table_name

In [ ]:
# Get the stream ARN
streams_client = boto3.client('dynamodbstreams')
response = streams_client.list_streams(
    TableName=model_instance_table_name
)

print response

stream = response['Streams'][0]['StreamArn']
print stream

## SMS Sub Context

In [ ]:
smssub_stack_name = stack_prefix + 'smssub'

In [ ]:
import boto3
client = boto3.client('cloudformation')

response = client.describe_stack_resources(
    StackName=smssub_stack_name
)


subs_tablename = ''

resources = response['StackResources']
for r in resources:
    rid = r['LogicalResourceId']
    
    if rid == 'SMSSubTable':
        subs_tablename = r['PhysicalResourceId']
        
print subs_tablename

## Deploy Lambda

In [ ]:
import os
cf_bucket = 'ds-status-us-west-2'
template_name = 'smspub.yml'
sms_stack = stack_prefix + 'smspub'
sms_zip = 'smspub-deployment.zip'

In [ ]:
%%bash -s "$template_name" "$sms_zip" "$cf_bucket"
make
aws s3 cp ./$1 s3://$3
aws s3 cp ./$2 s3://$3

In [ ]:
def form_s3_url_prefix(region):
    prefix = ''
    if region == 'us-east-1':
        prefix = 'https://s3.amazonaws.com'
    else:
        prefix = 'https://s3-' + region + '.amazonaws.com'
    return prefix

In [ ]:
import os
region = os.environ['AWS_DEFAULT_REGION']
bucketRoot = form_s3_url_prefix(region) + '/' + cf_bucket
print bucketRoot

In [ ]:
response = client.create_stack(
    StackName=sms_stack,
    TemplateURL=bucketRoot + '/' + template_name,
    Parameters=[
        {
            'ParameterKey': 'CodeBucketName',
            'ParameterValue': cf_bucket
        },
        {
            'ParameterKey': 'StreamArn',
            'ParameterValue': stream
        },
        {
            'ParameterKey': 'TableName',
            'ParameterValue': model_instance_table_name
        },
        {
            'ParameterKey': 'SMSSubscriptionsTableName',
            'ParameterValue': subs_tablename
        }
    ],
    Capabilities=[
        'CAPABILITY_IAM',
    ]
)

print response

In [ ]:
# Wait for it...
print 'wait for create of {}'.format(sms_stack)
waiter = client.get_waiter('stack_create_complete')
waiter.wait(
    StackName=sms_stack
)

print 'stack created'